Updated notebook - main changes: revert to separating each category into its own collection, implement 2 step pipeline (categorize, then retrieve from those relevant collections). Somehow, takes a substantial amount of time now (ie 7 queries in 17 minutes) (using general qa set and some other random set of data), to discuss.

Install some dependencies

In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

Get the Model You Want

In [ ]:
!pip install llama-cpp-python

Define Variables

In [2]:
from llama_cpp import Llama
from langchain_community.llms import HuggingFaceEndpoint
import os
from transformers import pipeline

model_path = "mistral-7b-instruct-v0.2.Q4_0.gguf"
model = Llama(model_path=model_path, n_ctx=2048, n_threads=8, verbose=False)

/opt/anaconda3/envs/CampusConnect/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kern

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")


/var/folders/fz/m3b664vj5hq4kdmwr645k9_h0000gn/T/ipykernel_18904/1223792325.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Define Data Sources

In [4]:
import pandas as pd
import concurrent.futures
import uuid
import os

file_names = [
    "study_permit_general", "work_permit_student_general", "work-study-data-llm",
    "vancouver_transit_qa_pairs", "permanent_residence_student_general", "data-with-sources",
    "faq_qa_pairs_general", "hikes_qa", "sfu-faq-with-sources", "sfu-housing-with-sources",
    "sfu-immigration-faq", "park_qa_pairs-up", "cultural_space_qa_pairs_up",
    "qa_pairs_food", "qa_pairs_year_and_month_avg", "qa_pairs_sfu_clubs"
]

collections = {}
batch_size = 32

def process_file(file):
    try:
        path = f'../Data/{file}.csv'
        if not os.path.exists(path):
            return f"{file} skipped (file not found)."

        df = pd.read_csv(path, usecols=lambda col: col.lower() in {"question", "answer"})
        df.columns = df.columns.str.lower()

        if "question" not in df.columns or "answer" not in df.columns:
            return f"{file} skipped (missing question/answer columns)."

        df = df.drop_duplicates(subset="question")
        df["text"] = df["question"].fillna('') + ' ' + df["answer"].fillna('')
        unique_texts = list(set(df["text"].dropna().tolist()))

        collection = client.get_or_create_collection(name=file)
        for i in range(0, len(unique_texts), batch_size):
            batch = unique_texts[i:i + batch_size]
            embeddings = embedding_model.embed_documents(batch)
            ids = [str(uuid.uuid4()) for _ in batch]
            collection.add(ids=ids, embeddings=embeddings, documents=batch)

        collections[file] = collection
        return f"{file}: Loaded {len(unique_texts)} docs."
    except Exception as e:
        return f"{file}: Error - {e}"

# parallelogram
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    results = list(executor.map(process_file, file_names))

for result in results:
    print(result)


study_permit_general: Loaded 14 docs.
work_permit_student_general: Loaded 23 docs.
work-study-data-llm: Loaded 178 docs.
vancouver_transit_qa_pairs: Loaded 37 docs.
permanent_residence_student_general: Loaded 6 docs.
data-with-sources: Loaded 75 docs.
faq_qa_pairs_general: Loaded 220 docs.
hikes_qa: Loaded 278 docs.
sfu-faq-with-sources: Loaded 102 docs.
sfu-housing-with-sources: Loaded 74 docs.
sfu-immigration-faq: Loaded 83 docs.
park_qa_pairs-up: Loaded 216 docs.
cultural_space_qa_pairs_up: Loaded 560 docs.
qa_pairs_food: Loaded 38 docs.
qa_pairs_year_and_month_avg: Loaded 1488 docs.
qa_pairs_sfu_clubs: Loaded 166 docs.


In [5]:
# define cat to collection mapping
# motivation: takes wayyyy too long now -> dataset size trippled and time grew exponentially...
# now takes around 2 mins on avg for each response..compared to 40 seconds previously..

collection_map = {
    "study": "study_permit_general",
    "student work": "work_permit_student_general",
    "work-study": "work-study-data-llm",
    "transit": "vancouver_transit_qa_pairs",
    "permanent residence": "permanent_residence_student_general",
    "general info": "data-with-sources",
    "faq": "faq_qa_pairs_general",
    "hiking": "hikes_qa",
    "sfu faq": "sfu-faq-with-sources",
    "housing": "sfu-housing-with-sources",
    "immigration": "sfu-immigration-faq",
    "parks": "park_qa_pairs-up",
    "culture": "cultural_space_qa_pairs_up",
    "food": "qa_pairs_food",
    "weather": "qa_pairs_year_and_month_avg",
    "clubs": "qa_pairs_sfu_clubs"
}


Function to now match for releveant document

In [6]:
def get_relevant_documents(query, categories, n_results=3):
    all_results = []
    query_embedding = embedding_model.embed_documents([query])[0]

    for category in categories:
        collection_name = collection_map[category]
        if collection_name in collections:
            try:
                result = collections[collection_name].query(
                    query_embeddings=[query_embedding],
                    n_results=n_results
                )
                docs = result.get("documents", [[]])[0]
                sims = result.get("distances", [[]])[0]

                all_results.extend(zip(docs, sims))
            except Exception as e:
                print(f"error querying {collection_name}: {e}")

    all_results = sorted(all_results, key=lambda x: x[1])

    return all_results[:n_results]


### Classify Prompt

In [7]:
import re
import difflib

valid_categories = list(collection_map.keys())
fallback_category = "faq"

def classify_query(query):
    category_prompt = f"""
    You are a classifier for a Q&A system for international students in British Columbia.
    Choose the **1 most relevant** category from this list, or at most 3 if absolutely needed (comma-separated):

    {", ".join(valid_categories)}

    Query: "{query}"

    Return only the category name(s) as a comma-separated string.
    """

    response = model(category_prompt, max_tokens=50, temperature=0.1)["choices"][0]["text"].strip().lower()
    print("Raw out:", response)

    tokens = re.findall(r'\b\w+\b', response)

    matched = []
    for token in tokens:
        closest = difflib.get_close_matches(token, valid_categories, n=1, cutoff=0.8)
        if closest and closest[0] not in matched:
            matched.append(closest[0])
        if len(matched) == 3:
            break

    if fallback_category not in matched:
      matched.append(fallback_category)

    return matched[:3]


Generate Answer

In [8]:
def generate_answer(query):
    categories = classify_query(query)
    print(f"Categories {categories}\n")
    relevant_documents = get_relevant_documents(query, categories)

    if not relevant_documents:
        return {
            "Response": "Sorry, no relevant documents found."
        }

    #relevant_documents = list(set(relevant_documents))

    seen = set()
    unique_docs = []
    for doc, sim in relevant_documents:
        if doc not in seen:
            seen.add(doc)
            unique_docs.append((doc, sim))

    print("Relevant Documents with Similarity Scores:")
    for doc, sim in unique_docs:
        print(f"Similarity: {sim:.4f}\nDoc: {doc}\n")

    relevant_texts = "\n\n".join([doc for doc, _ in unique_docs])

    rag_prompt = f"""
    You are a helpful, friendly assistant for international students new to British Columbia, Canada.

    Below are some reference documents that may be relevant to the user's question:
    {relevant_texts}

    INSTRUCTIONS:
    1. If the user's query is just a greeting (like "hello", "hi", "what's up"):
       - Respond with a single brief friendly greeting
       - Offer to help with questions about studying or living in BC
       - Do NOT include ANY information from the reference documents
       - Do NOT create additional questions and answers

    2. If the user is asking for information:
       - Be friendly and answer based ONLY on the reference documents if relevant
       - Include all relevant information from the documents
       - If the documents don't provide sufficient information, say "I don't have enough information to answer that. Please refer to official sources."

    3. IMPORTANT: Never generate additional content beyond answering the user's question

    User question: {query}

    Your response (just the answer, no preamble):
    """
    #    - Keep your answer brief and concise (under 30 words when possible)

    response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)["choices"][0]["text"]
    # response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)

    return {
        "Response": response_after_rag
    }


Example Usage

In [9]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")

In [13]:
answer = generate_answer("What is the upass BC for students in vancouver transit?")
print(answer["Response"])

Raw out: the upass bc is a student transit pass for vancouver, bc. therefore, the most relevant category is "transit".
Categories ['transit', 'faq']

Relevant Documents with Similarity Scores:
Similarity: 0.6942
Doc: How do I pay for the bus in Vancouver? In Vancouver, you pay for public transit using a Compass Card, which you can purchase at SkyTrain stations, London Drugs stores, and other locations. As an international student, you're likely eligible for the discounted U-Pass BC program through your institution.

 The U-Pass BC is a discounted transit pass program for full-time students at participating institutions in British Columbia. It allows students to travel on TransLink, BC Ferries, and other partner transit systems at a reduced rate. The cost and eligibility requirements may vary depending on your institution. For more information, please contact your school's student services or visit their website.


In [14]:
import pandas as pd
from evaluate import load
bertscore = load("bertscore")

benchmark_data = pd.read_csv("../p-eval-set/unseen-data.csv")

for idx, row in benchmark_data.iterrows():
    user_query = row["Question"]
    correct_answer = row["Answer"]

    responses = generate_answer(user_query)
    
    predictions = [responses.get("Response", "N/A")]
    references = [correct_answer]
    results = bertscore.compute(predictions=predictions, references=references, lang="en")

    print("\n" + "="*50)
    print(f"Benchmark Query {idx + 1}: {user_query}")
    print("="*50)
    print("\nRAG Response:\n", responses.get("Response", "N/A"))
    print("\n(Benchmark) Answer:\n", correct_answer)
    print("BERT Score == ", results)
    print("="*50 + "\n\n")


Raw out: category: transit
Categories ['transit', 'faq']

Relevant Documents with Similarity Scores:
Similarity: 0.8226
Doc: How do I get to Simon Fraser University (SFU) by bus? To get to Simon Fraser University (SFU), you can take bus routes 145 (leaving from Production Way University Station), R5 (leaving from Burrard Station), 143 (leaving from Burquitlam Station), 144 (leaving from Metrotown Station). Always check the TransLink website or mobile app for the most up-to-date information.



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Benchmark Query 1: How do I get to Metrotown via transit from SFU Burnaby?

RAG Response:
 
    To get to Metrotown from SFU Burnaby, you can take bus route 144 from the nearby SFU Burnaby campus. The bus stop is located at the corner of Gaglardi Way and University Drive. The journey takes approximately 20 minutes. Always check the TransLink website or mobile app for the most up-to-date information.

(Benchmark) Answer:
 There are 2 possible routes you can take to reach Metrotown: 1) Take the 145 bus to Production Way University station, hop on the SkyTrain (Expo Line) heading towards Waterfront Station, and ride the SkyTrain and get off at Metrotown Station, 2) or you can directly take the 144 bus to Metrotown station which includes about 45 stops (~1 hour).
BERT Score ==  {'precision': [0.8465505838394165], 'recall': [0.8508000373840332], 'f1': [0.8486699461936951], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.51.3)'}


Raw out: ----------------------------------

In [29]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["An SFU computing ID"]
references = ["An SFU Computing ID"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
print("Results == ", results)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Results ==  {'precision': [0.9849280714988708], 'recall': [0.9849280714988708], 'f1': [0.9849280714988708], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.51.3)'}
